In [ ]:
pip install 

In [ ]:
pip install networkx

In [ ]:
pip install dash_cytoscape

In [ ]:
n = 5000000

def reducir_lineas(input_file, output_file, n):
    with open(input_file, 'r') as file_in, open(output_file, 'w') as file_out:
        for i, linea in enumerate(file_in):
            if i < n:
                file_out.write(linea)
            else:
                break

# Ejemplo de uso
target = 'user'
reducir_lineas(f'10_million_{target}.txt', f'{n}_{target}.txt', n)

In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import networkx as nx
import random

# Crear el grafo vacío sin aristas
G = nx.DiGraph()

# Cargar las coordenadas desde el archivo .txt
def load_coordinates(file_path):
    coords = {}
    with open(file_path, 'r') as file:
        for idx, line in enumerate(file):
            x, y = map(float, line.strip().split(','))
            coords[idx + 1] = (x, y)  # Asigna el nodo en orden (1, 2, 3, ...)
    return coords

# Cargar los vecinos desde el archivo .txt
def load_neighbors(file_path):
    neighbors = {}
    with open(file_path, 'r') as file:
        for idx, line in enumerate(file, start=1):  # Usar enumerate para obtener el índice
            parts = list(map(int, line.strip().replace(',', '').split()))
            neighbors[idx] = parts  # Asociar la línea con su índice como nodo
    return neighbors

# Ruta de los archivos con las coordenadas y los vecinos
coords_file_path = '10_million_location.txt'
neighbors_file_path = '10_million_user.txt'

coords = load_coordinates(coords_file_path)
neighbors = load_neighbors(neighbors_file_path)

# Añadir nodos y coordenadas sin aristas
for node, (x, y) in coords.items():
    G.add_node(node, pos=(x, y))

# Añadir aristas basadas en los vecinos
for node, nbrs in neighbors.items():
    if node in coords:
        for nbr in nbrs:
            if nbr in coords:
                if not G.has_edge(nbr, node):  # Si no existe la inversa, agregar la arista
                    G.add_edge(node, nbr, weight=1)  # Asignar pesos aleatorios

# Detectar componentes débilmente conectadas
weakly_connected = list(nx.weakly_connected_components(G))

# Función para generar colores únicos
def assign_colors(components):
    component_colors = []
    for _ in components:
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        component_colors.append(color)
    return component_colors

# Generar colores para las componentes
component_colors = assign_colors(weakly_connected)

# Configuración de la aplicación Dash
app = dash.Dash(__name__)

# Layout inicial con combobox
app.layout = html.Div([
    dcc.Dropdown(
        id='component-selector',
        options=[{'label': f'Componente {i+1} ({len(comp)} nodos)', 'value': i} for i, comp in enumerate(weakly_connected)],
        placeholder="Seleccione una componente",
        style={'width': '50%'}
    ),
    dcc.Graph(
        id='graph',
        config={'scrollZoom': True},
        style={'height': '75vh'}
    ),
    html.Div(id='analysis-output', style={'margin-top': '20px'})  # Salida del análisis
])

# Callback para actualizar el gráfico según la selección
@app.callback(
    [Output('graph', 'figure'), Output('analysis-output', 'children')],
    [Input('component-selector', 'value')]
)
def update_graph(selected_component):
    if selected_component is None:
        return go.Figure(), "Seleccione una componente para análisis."  # Gráfico vacío y mensaje

    # Obtener la componente seleccionada
    selected_nodes = weakly_connected[selected_component]
    subgraph = G.subgraph(selected_nodes).copy()

    # Extraer posiciones y colores para los nodos
    positions = nx.get_node_attributes(subgraph, 'pos')
    x_values = [positions[node][0] for node in subgraph.nodes()]
    y_values = [positions[node][1] for node in subgraph.nodes()]
    colors = [component_colors[selected_component]] * len(subgraph.nodes())

    # Crear los trazos para nodos
    node_trace = go.Scattergl(
        x=x_values,
        y=y_values,
        mode='markers',
        marker=dict(size=5, color=colors),
        hoverinfo='text',
        hovertext=[f'Node {node}' for node in subgraph.nodes()],
    )

    # Crear los trazos para aristas
    edge_x = []
    edge_y = []
    for edge in subgraph.edges():
        x0, y0 = positions[edge[0]]
        x1, y1 = positions[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scattergl(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color='gray'),
        mode='lines',
        hoverinfo='none',
    )

    # Cálculo del camino más corto promedio
    try:
        avg_shortest_path_length = nx.average_shortest_path_length(subgraph)
    except nx.NetworkXError:
        avg_shortest_path_length = "Infinito (Grafo no fuertemente conectado)"

    # Árbol de expansión mínima
    try:
        mst = nx.minimum_spanning_tree(subgraph.to_undirected(), weight='weight')
        mst_edges = list(mst.edges(data=True))
        mst_total_weight = sum(edge[2]['weight'] for edge in mst_edges)
    except Exception as e:
        mst_total_weight = "N/A (No se puede calcular MST)"

    # Configurar layout del gráfico
    layout = go.Layout(
        title=f"Componente {selected_component + 1}",
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
    )

    # Información adicional
    analysis_text = [
        html.P(f"Camino más corto promedio: {avg_shortest_path_length}"),
        html.P(f"Peso total del Árbol de Expansión Mínima: {mst_total_weight}")
    ]

    return go.Figure(data=[edge_trace, node_trace], layout=layout), analysis_text

if __name__ == '__main__':
    app.run_server(debug=False)